In [1]:
import pandas as pd
import numpy as np
from pandas import json

In [2]:
cg_data = pd.read_csv('campgrounds.csv')

In [3]:
cg_data.shape

(11, 6)

In [4]:
cg_data.head()

facilityname  facilitylatitude  facilitylongitude flush  \
0                      Mazama         42.865508        -122.166149     1   
1  Claude Lewis Sno-Park-USFS         43.063217        -122.342876    \N   
2                  Lost Creek         42.879145        -122.037881     1   
3                     Hamaker         43.056641        -122.330322    \N   
4        Huckleberry Mountain         42.877807        -122.337174    \N   

  shower vault  
0      1    \N  
1     \N     1  
2     \N    \N  
3     \N     1  
4     \N     1

In [7]:
cg_data_clean = cg_data

In [11]:
cg_data_clean = cg_data_clean.replace({'flush': {'1': 'Flush toilet', '0':'', '\\N':''}})
cg_data_clean = cg_data_clean.replace({'shower': {'1': 'Shower', '0':'', '\\N':''}})
cg_data_clean = cg_data_clean.replace({'vault': {'1': 'Vault toilet', '0':'', '\\N':''}})

In [12]:
cg_data_clean = cg_data_clean.rename(columns={'facilityname': 'title',
                                             'facilitylatitude': 'latitude',
                                             'facilitylongitude': 'longitude'})

In [13]:
cg_data_clean

title   latitude   longitude         flush  \
0                           Mazama  42.865508 -122.166149  Flush toilet   
1       Claude Lewis Sno-Park-USFS  43.063217 -122.342876                 
2                       Lost Creek  42.879145 -122.037881  Flush toilet   
3                          Hamaker  43.056641 -122.330322                 
4             Huckleberry Mountain  42.877807 -122.337174                 
5             Natural Bridge -USFS  42.889648 -122.463867                 
6                    Farewell Bend  42.919678 -122.433105                 
7                      Scott Creek  42.884588 -121.924772                 
8   Thousand Springs Sno-Park-USFS  42.913045 -122.324644                 
9                      Union Creek  42.910278 -122.450000                 
10        Farewell Bend Campground  42.916239 -122.435331                 

    shower         vault  
0   Shower                
1           Vault toilet  
2                         
3           Vault toilet  
4           Vault toilet  
5           Vault toilet  
6                         
7           Vault toilet  
8           Vault toilet  
9           Vault toilet  
10

In [14]:
cg_data_clean['description'] = cg_data_clean[['flush','shower','vault']].apply(lambda x: ', '.join(x), axis=1)

In [17]:
def clean_description(description):
    description = description.strip()
    while((description.startswith(',') or description.endswith(',')) and len(description) > -1):
        if description.endswith(','):
            description = description[0:len(description)-1]
        if description.startswith(','):
            description = description[1:len(description)]
        description = description.strip()
    return description

In [18]:
cg_data_clean['description'] = cg_data_clean.description.apply(lambda x: clean_description(x))

In [19]:
cg_data_clean

title   latitude   longitude         flush  \
0                           Mazama  42.865508 -122.166149  Flush toilet   
1       Claude Lewis Sno-Park-USFS  43.063217 -122.342876                 
2                       Lost Creek  42.879145 -122.037881  Flush toilet   
3                          Hamaker  43.056641 -122.330322                 
4             Huckleberry Mountain  42.877807 -122.337174                 
5             Natural Bridge -USFS  42.889648 -122.463867                 
6                    Farewell Bend  42.919678 -122.433105                 
7                      Scott Creek  42.884588 -121.924772                 
8   Thousand Springs Sno-Park-USFS  42.913045 -122.324644                 
9                      Union Creek  42.910278 -122.450000                 
10        Farewell Bend Campground  42.916239 -122.435331                 

    shower         vault           description  
0   Shower                Flush toilet, Shower  
1           Vault toilet          Vault toilet  
2                                 Flush toilet  
3           Vault toilet          Vault toilet  
4           Vault toilet          Vault toilet  
5           Vault toilet          Vault toilet  
6                                               
7           Vault toilet          Vault toilet  
8           Vault toilet          Vault toilet  
9           Vault toilet          Vault toilet  
10

In [20]:
geojson_df = cg_data_clean[['title','latitude','longitude','description']]

In [21]:
geojson_df

title   latitude   longitude  \
0                           Mazama  42.865508 -122.166149   
1       Claude Lewis Sno-Park-USFS  43.063217 -122.342876   
2                       Lost Creek  42.879145 -122.037881   
3                          Hamaker  43.056641 -122.330322   
4             Huckleberry Mountain  42.877807 -122.337174   
5             Natural Bridge -USFS  42.889648 -122.463867   
6                    Farewell Bend  42.919678 -122.433105   
7                      Scott Creek  42.884588 -121.924772   
8   Thousand Springs Sno-Park-USFS  42.913045 -122.324644   
9                      Union Creek  42.910278 -122.450000   
10        Farewell Bend Campground  42.916239 -122.435331   

             description  
0   Flush toilet, Shower  
1           Vault toilet  
2           Flush toilet  
3           Vault toilet  
4           Vault toilet  
5           Vault toilet  
6                         
7           Vault toilet  
8           Vault toilet  
9           Vault toilet  
10

In [22]:
collection = {'type':'FeatureCollection', 'features':[]}

In [24]:
def feature_from_row(title, latitude, longitude, description):
    feature = {'type': 'Feature',
              'properties': {'title': '', 'description': ''},
              'geometry': {'type': 'Point', 'coordinates': []}
              }
    feature['geometry']['coordinates'] = [longitude, latitude]
    feature['properties']['title'] = title
    feature['properties']['description'] = description
    collection['features'].append(feature)
    return feature

In [25]:
geojson_series = geojson_df.apply(lambda x: feature_from_row(x['title'],x['latitude'],x['longitude'],x['description']),
                                 axis=1)

In [26]:
collection

{'features': [{'geometry': {'coordinates': [-122.166149, 42.865508],
    'type': 'Point'},
   'properties': {'description': 'Flush toilet, Shower', 'title': 'Mazama'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.342876, 43.063217], 'type': 'Point'},
   'properties': {'description': 'Vault toilet',
    'title': 'Claude Lewis Sno-Park-USFS'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.037881, 42.879145], 'type': 'Point'},
   'properties': {'description': 'Flush toilet', 'title': 'Lost Creek'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.330322, 43.056641], 'type': 'Point'},
   'properties': {'description': 'Vault toilet', 'title': 'Hamaker'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.337174, 42.877807], 'type': 'Point'},
   'properties': {'description': 'Vault toilet',
    'title': 'Huckleberry Mountain'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.463867, 42.889648], 'type': 'Point'},
   'properties': {'

In [27]:
with open('collection.geojson', 'w') as outfile:
    json.dump(collection, outfile)

In [28]:
test = pd.read_json('collection.geojson')

In [29]:
test

features               type
0   {'properties': {'description': 'Flush toilet, ...  FeatureCollection
1   {'properties': {'description': 'Vault toilet',...  FeatureCollection
2   {'properties': {'description': 'Flush toilet',...  FeatureCollection
3   {'properties': {'description': 'Vault toilet',...  FeatureCollection
4   {'properties': {'description': 'Vault toilet',...  FeatureCollection
5   {'properties': {'description': 'Vault toilet',...  FeatureCollection
6   {'properties': {'description': '', 'title': 'F...  FeatureCollection
7   {'properties': {'description': 'Vault toilet',...  FeatureCollection
8   {'properties': {'description': 'Vault toilet',...  FeatureCollection
9   {'properties': {'description': 'Vault toilet',...  FeatureCollection
10  {'properties': {'description': '', 'title': 'F...  FeatureCollection